# simulating the stroke unit

In [1]:
import numpy as np
import pandas as pd
import random 
import itertools
import math
import matplotlib.pyplot as plt

In [2]:
import simpy
simpy.__version__

'4.1.1'

# distribution classes

distribution classes used
- exponential for IAT (inter arrival time)
- lognormal for length of stay

In [3]:
class Exponential:
    '''
    Convenience class for the exponential distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, mean, random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        mean: float
            The mean of the exponential distribution
        
        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rand = np.random.default_rng(seed=random_seed)
        self.mean = mean
        
    def sample(self, size=None):
        '''
        Generate a sample from the exponential distribution
        
        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rand.exponential(self.mean, size=size)

class Lognormal:
    """
    Encapsulates a lognormal distirbution
    """
    def __init__(self, mean, stdev, random_seed=None):
        """
        Params:
        -------
        mean = mean of the lognormal distribution
        stdev = standard dev of the lognormal distribution
        """
        self.rand = np.random.default_rng(seed=random_seed)
        mu, sigma = self.normal_moments_from_lognormal(mean, stdev**2)
        self.mu = mu
        self.sigma = sigma
        
    def normal_moments_from_lognormal(self, m, v):
        '''
        Returns mu and sigma of normal distribution
        underlying a lognormal with mean m and variance v
        source: https://blogs.sas.com/content/iml/2014/06/04/simulate-lognormal
        -data-with-specified-mean-and-variance.html

        Params:
        -------
        m = mean of lognormal distribution
        v = variance of lognormal distribution
                
        Returns:
        -------
        (float, float)
        '''
        phi = math.sqrt(v + m**2)
        mu = math.log(m**2/phi)
        sigma = math.sqrt(math.log(phi**2/m**2))
        return mu, sigma
        
    def sample(self):
        """
        Sample from the normal distribution
        """
        return self.rand.lognormal(self.mu, self.sigma)

class Bernoulli:
    '''
    Convenience class for the Bernoulli distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, p, random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        p: float
            probability of drawing a 1
        
        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rand = np.random.default_rng(seed=random_seed)
        self.p = p
        
    def sample(self, size=None):
        '''
        Generate a sample from the exponential distribution
        
        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rand.binomial(n=1, p=self.p, size=size)


# utility function 

In [4]:
def trace(msg):
    '''
    Utility function for printing simulation
    set the TRACE constant to FALSE to 
    turn tracing off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

# model parameters

In [5]:
# resource counts
N_BEDS = 10

# time between arrivals in days (exponential)
MEAN_IAT_STROKE = 1.2
MEAN_IAT_TIA = 9.3
MEAN_IAT_CN = 3.6
MEAN_IAT_OTHER = 3.2

# acute stroke unit length of stay in days (lognormal)
# acute stroke unit length of stay (lognormal distribution) for each patient type
STAY_PARAMS = {
    "Stroke": (7.4, 8.6),         # Mean = 7.4, SD = 8.6
    "TIA": (1.8, 2.3),            # Mean = 1.8, SD = 2.3
    "ComplexNeuro": (4.0, 5.0),   # Mean = 4.0, SD = 5.0
    "Other": (3.8, 5.2)           # Mean = 3.8, SD = 5.2
}
# transfer (bernoulli)
PROB_ESD = 0.13

# SEEDS to reproduce results of a single run
REPRODUCIBLE_RUN = True

if REPRODUCIBLE_RUN:
    SEEDS = [42, 101, 1066, 1966, 2013, 999, 1444, 2016]
else:
    SEEDS = [None, None, None, None, None, None, None, None]

In [6]:
class Scenario:
    '''
    Parameter container class for acute stroke unit model.
    '''
    def __init__(self, 
                 iat_stroke = MEAN_IAT_STROKE,
                 iat_tia = MEAN_IAT_TIA,
                 iat_cn = MEAN_IAT_CN,
                 iat_other = MEAN_IAT_OTHER, 
                 name=None):
        '''
        The init method sets up our defaults. 
        
        Params:
        -------
        
        name - str or None
            optional name for scenario
        '''
        
        # optional name
        self.name = name
        
        # beds
        self.unit_beds = N_BEDS

        #store the parameters 
        self.iat_stroke = iat_stroke
        self.iat_tia = iat_tia
        self.iat_cn = iat_cn
        self.iat_other = iat_other

        #initialise results to zero
        self.init_results_variables()
        #Initialise patient arrival distribution
        self.init_sampling()
        self.patients = []
        
        # Length of stay distribution (lognormal) for each patient type using STAY_PARAMS
        self.length_of_stay_dist = {
            patient_type: Lognormal(mean, std, random_seed=SEEDS[4])  
            for patient_type, (mean, std) in STAY_PARAMS.items()
        }

        # ESD transfer: prob that patient goes to ESD.
        self.esd_transfer = Bernoulli(PROB_ESD, random_seed=SEEDS[5])
        
    def init_sampling(self):
    # inter-arrival distribution for different patient types
        self.arrival_dist = {
            "Stroke": Exponential(MEAN_IAT_STROKE, random_seed=SEEDS[0]),
            "TIA": Exponential(MEAN_IAT_TIA, random_seed=SEEDS[1]),
            "ComplexNeuro": Exponential(MEAN_IAT_CN, random_seed=SEEDS[2]),
            "Other": Exponential(MEAN_IAT_OTHER, random_seed=SEEDS[3]),
        }

    def init_results_variables(self):
        self.results = {}
        self.results["n_stroke"] = 0
        self.results["n_tia"] = 0
        self.results["n_cn"] = 0
        self.results["n_other"] = 0
            

# build the model 

## model the unit
## Arrival Generator per patient_type
Function which creates an arrival generator per patient type. Therefore, 4 functions for each patient entering the Acute Stroke Unit.

!!! Note to self: code redundancy can be fixed by creating one arrival generator but this is better to do after making sure the model runs properly.

In [7]:
class AcuteStrokeUnit:  
    '''
    Model of an acute stroke unit
    '''
    def __init__(self, env, args):
        '''
        Contructor
        
        Params:
        -------
        env: simpy.Environment
        
        args: Scenario
            container class for simulation model inputs.
        '''
        self.env = env
        self.args = args 
        self.init_model_resources(args)
        self.patients = []

        # 🔹 Add a shared patient ID counter
        self.patient_id_counter = 0  

        # Start the arrival processes for different patient types
        env.process(self.stroke_arrivals_generator())
        env.process(self.tia_arrivals_generator())
        env.process(self.complexneuro_arrivals_generator())
        env.process(self.other_arrivals_generator())

        
    def init_model_resources(self, args):
        '''
        Setup the simpy resource objects
        
        Params:
        ------
        args - Scenario
            Simulation Parameter Container
        '''
        args.unit_beds = simpy.Resource(self.env, 
                                          capacity=args.unit_beds)  

    def start_patient_arrivals(self):
        '''Start arrival processes for all patient types'''
        self.env.process(self.stroke_arrivals_generator())
        self.env.process(self.tia_arrivals_generator())
        self.env.process(self.complexneuro_arrivals_generator())
        self.env.process(self.other_arrivals_generator())

    def stroke_arrivals_generator(self):
        """Arrival process for stroke patients."""
        while True:
            inter_arrival_time = self.args.arrival_dist["Stroke"].sample()
            yield self.env.timeout(inter_arrival_time)

            # 🔹 Increment global counter
            self.patient_id_counter += 1  
        
            new_patient = AcutePatient(self.patient_id_counter, self.env, self.args, patient_type="Stroke")
            self.patients.append(new_patient)
            self.env.process(new_patient.assessment())
            trace(f'Patient {self.patient_id_counter} STROKE arrival at: {self.env.now:.3f}')
            
    def tia_arrivals_generator(self):
        """Arrival process for TIA patients."""
        while True:
            inter_arrival_time = self.args.arrival_dist["TIA"].sample()
            yield self.env.timeout(inter_arrival_time)
    
            self.patient_id_counter += 1  
            new_patient = AcutePatient(self.patient_id_counter, self.env, self.args, patient_type="TIA")
            self.patients.append(new_patient)
            self.env.process(new_patient.assessment())
    
            trace(f'Patient {self.patient_id_counter} TIA arrival at: {self.env.now:.3f}')

    def complexneuro_arrivals_generator(self):
        """Arrival process for ComplexNeuro patients."""
        while True:
            inter_arrival_time = self.args.arrival_dist["ComplexNeuro"].sample()
            yield self.env.timeout(inter_arrival_time)
    
            self.patient_id_counter += 1  
            new_patient = AcutePatient(self.patient_id_counter, self.env, self.args, patient_type="ComplexNeuro")
            self.patients.append(new_patient)
            self.env.process(new_patient.assessment())
    
            trace(f'Patient {self.patient_id_counter} ComplexNeuro arrival at: {self.env.now:.3f}')
    
    def other_arrivals_generator(self):
        """Arrival process for Other patients."""
        while True:
            inter_arrival_time = self.args.arrival_dist["Other"].sample()
            yield self.env.timeout(inter_arrival_time)
    
            self.patient_id_counter += 1  
            new_patient = AcutePatient(self.patient_id_counter, self.env, self.args, patient_type="Other")
            self.patients.append(new_patient)
            self.env.process(new_patient.assessment())
    
            trace(f'Patient {self.patient_id_counter} Other arrival at: {self.env.now:.3f}')

    def run_summary_frame(self):  # 👈 ADD THIS HERE
            ''' Generate a summary of patient statistics. '''
            if not self.patients:
                return None  # No patients processed
            
            import pandas as pd
    
            # Collecting basic statistics
            data = {
                "Patient Type": [p.patient_type for p in self.patients],
                "Length of Stay": [p.length_of_stay for p in self.patients],
                "Time in System": [p.time_in_system for p in self.patients],
                "Time to Bed": [p.time_to_bed for p in self.patients]
            }
    
            return pd.DataFrame(data)  # Return as a DataFrame

## model the patient

In [8]:
class AcutePatient:
    '''
    Patient in the minor ED process
    '''
    def __init__(self, identifier, env, args, patient_type):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        args: Scenario
            The input data for the scenario
        '''
        # patient id and environment
        self.identifier = identifier
        self.env = env
        self.patient_type = patient_type  # Track patient category
        
        # Get length of stay distribution from Scenario
        self.length_of_stay_dist = args.length_of_stay_dist[patient_type]

        # stroke unit parameters
        self.unit_beds = args.unit_beds
        
        # esd transfer: prob that patient is transfered to ESD.
        self.esd_transfer = args.esd_transfer
                
        # individual patient metrics
        self.time_to_bed = 0.0
        self.length_of_stay = 0.0
        self.time_to_esd = 0.0
        self.time_in_system = 0.0
        self.four_hour_target = 0.0
    
    def assessment(self):
        '''
        simulates the process for acute stroke unit
        
        1. Patient enters the system and requests a bed
        2. Wait for bed availability and simulate length of stay
        3. Calculate time in system, ESD transfer, and if the 4-hour target is met
        
        '''
        # record the time that patient entered the system
        arrival_time = self.env.now
    
        # Request a bed from the stroke unit
        with self.unit_beds.request() as req:
            yield req
            
            # Record when the patient actually gets a bed
            bed_time = self.env.now  # Record when the patient gets the bed
            trace(f'Patient {self.identifier} ({self.patient_type}) gets a bed at {self.env.now:.3f}')
            
            # Calculate time to bed (how long the patient waited for a bed)
            self.time_to_bed = self.env.now - arrival_time
            
           # Simulate the patient's length of stay in the acute unit
            length_of_stay = self.length_of_stay_dist.sample()
            
            discharge_time = bed_time + length_of_stay  # When they leave
            
            # Simulate the time the patient spends in the unit
            yield self.env.timeout(length_of_stay)
            
            # Now the patient is leaving
            self.time_in_system = self.env.now - arrival_time  # Total time from arrival to departure
            self.length_of_stay = self.env.now - bed_time  # New length of stay calculation            
            
            # Log the patient's departure and the total time in the system
            trace(f'Patient {self.identifier} departs at {self.env.now:.3f}; '
              f'Length of stay: {self.length_of_stay:.3f} days; '
              f'Time in system: {self.time_in_system:.3f} days')
            
            # Check if the patient met the 4-hour target (240 minutes)
            if self.time_in_system <= (4 * 60):  # 4 hours = 240 minutes
                self.four_hour_target = 1  # Patient met the target
            else:
                self.four_hour_target = 0  # Patient exceeded the target
            
            # Check if the patient is transferred to the ESD (Early Supported Discharge)
            if self.esd_transfer.sample():
                self.time_to_esd = self.env.now
                trace(f'Patient {self.identifier} transferred to ESD at {self.env.now:.3f}')
            
            # Final discharge time log
            trace(f'Patient {self.identifier} discharged at {self.env.now:.3f}')

# script to run the model 

In [9]:
# run length in minutes
RUN_LENGTH = 100

# Define trace as on/off
#TRACE = False
TRACE = True 

# create simpy environment
env = simpy.Environment()

# base case scenario with default parameters
default_args = Scenario()
# reset all results variables to zero and empty
default_args.init_results_variables()

# create the model
model = AcuteStrokeUnit(env, default_args)

# we pass all arrival generators to simpy 
env.process(model.stroke_arrivals_generator())  # Calling as a method of the model instance
env.process(model.tia_arrivals_generator())    # Calling as a method of the model instance
env.process(model.complexneuro_arrivals_generator())  # Calling as a method of the model instance
env.process(model.other_arrivals_generator())  # Calling as a method of the model instance



env.run(until=RUN_LENGTH)
print(f'end of run. simulation clock time = {env.now}')
scenario = Scenario()  # Create an instance
print(scenario.results)  # Access and print the results dictionary



Patient 1 Other arrival at: 1.333
Patient 1 (Other) gets a bed at 1.333
Patient 2 Other arrival at: 1.569
Patient 2 (Other) gets a bed at 1.569
Patient 3 Other arrival at: 2.721
Patient 3 (Other) gets a bed at 2.721
Patient 4 STROKE arrival at: 2.803
Patient 4 (Stroke) gets a bed at 2.803
Patient 1 departs at 2.826; Length of stay: 1.494 days; Time in system: 1.494 days
Patient 1 discharged at 2.826
Patient 5 STROKE arrival at: 2.885
Patient 5 (Stroke) gets a bed at 2.885
Patient 6 STROKE arrival at: 3.221
Patient 6 (Stroke) gets a bed at 3.221
Patient 7 STROKE arrival at: 3.325
Patient 7 (Stroke) gets a bed at 3.325
Patient 8 Other arrival at: 3.388
Patient 8 (Other) gets a bed at 3.388
Patient 9 ComplexNeuro arrival at: 4.199
Patient 9 (ComplexNeuro) gets a bed at 4.199
Patient 8 departs at 4.373; Length of stay: 0.986 days; Time in system: 0.986 days
Patient 8 discharged at 4.373
Patient 10 ComplexNeuro arrival at: 4.530
Patient 10 (ComplexNeuro) gets a bed at 4.530
Patient 2 depart

## Running metrics


In [10]:
# check between minutes and days 

In [11]:
discharged_patients = [p for p in model.patients if hasattr(p, 'length_of_stay') and p.length_of_stay > 0]

mean_length_of_stay = np.mean([p.length_of_stay for p in discharged_patients])

print(f"Mean Length of Stay: {mean_length_of_stay:.2f} days")


Mean Length of Stay: 4.93 days


In [12]:
# Ensure there are patients in the system
if model.patients:

    # Group patients by their type (e.g., "Stroke", "TIA", etc.)
    patient_types = set(p.patient_type for p in model.patients)  # Get unique patient types
    for patient_type in patient_types:
        # Filter patients by type
        patients_by_type = [p for p in model.patients if p.patient_type == patient_type]

        # 1. Mean time in system (from arrival to discharge)
        mean_time_in_system = np.mean([p.time_in_system for p in patients_by_type])

        # 2. Mean length of stay (from bed assignment to discharge)
        discharged_patients = [p for p in patients_by_type if hasattr(p, 'length_of_stay') and p.length_of_stay > 0]
        mean_length_of_stay = np.mean([p.length_of_stay for p in discharged_patients]) if discharged_patients else 0

        # 3. Proportion of patients who met the 4-hour target
        four_hours = np.mean([p.four_hour_target for p in patients_by_type])

        # 4. Mean time to bed (arrival → bed assignment)
        mean_time_to_bed = np.mean([p.time_to_bed for p in patients_by_type])

        # 5. Mean time to ESD transfer (bed → ESD transfer, only for transferred patients)
        patients_with_esd = [p for p in patients_by_type if p.time_to_esd > 0]
        mean_time_to_esd = np.mean([p.time_to_esd - p.time_to_bed for p in patients_with_esd]) if patients_with_esd else 0

        # Print results for this patient type
        print(f'\nResults for {patient_type} Patients\n------------------')
        print(f'Mean Time in System (days): {mean_time_in_system:.2f}')
        print(f'Mean Length of Stay (days): {mean_length_of_stay:.2f}')  # ✅ Corrected Calculation
        print(f'Mean Time to Bed (days): {mean_time_to_bed:.2f}')
        print(f'Mean Time to ESD (days): {mean_time_to_esd:.2f}' if patients_with_esd else "No ESD transfers")
        print(f'Proportion Discharged Before 4 Hours: {four_hours:.2f}')
else:
    print("No patients recorded in this simulation run.")



Results for Other Patients
------------------
Mean Time in System (days): 12.41
Mean Length of Stay (days): 3.21
Mean Time to Bed (days): 10.23
Mean Time to ESD (days): 43.78
Proportion Discharged Before 4 Hours: 0.68

Results for ComplexNeuro Patients
------------------
Mean Time in System (days): 10.34
Mean Length of Stay (days): 2.86
Mean Time to Bed (days): 10.00
Mean Time to ESD (days): 25.14
Proportion Discharged Before 4 Hours: 0.53

Results for Stroke Patients
------------------
Mean Time in System (days): 14.13
Mean Length of Stay (days): 6.41
Mean Time to Bed (days): 11.75
Mean Time to ESD (days): 42.08
Proportion Discharged Before 4 Hours: 0.61

Results for TIA Patients
------------------
Mean Time in System (days): 12.15
Mean Length of Stay (days): 1.37
Mean Time to Bed (days): 11.24
Mean Time to ESD (days): 10.93
Proportion Discharged Before 4 Hours: 0.67


In [13]:
# new


In [14]:
import pandas as pd
import numpy as np
import simpy

def summarize_results(model):
    """ Generate a summary DataFrame with mean results for each patient type. """

    if not model.patients:
        print("⚠ No patients recorded in this simulation run.")
        return pd.DataFrame()  # Return an empty DataFrame

    patient_types = set(p.patient_type for p in model.patients)  # Unique patient types
    summary_data = {}

    for patient_type in patient_types:
        patients_by_type = [p for p in model.patients if p.patient_type == patient_type]

        # Compute mean length of stay (only for patients with a valid length_of_stay)
        discharged_patients = [p for p in patients_by_type if hasattr(p, 'length_of_stay') and p.length_of_stay > 0]
        mean_length_of_stay = np.mean([p.length_of_stay for p in discharged_patients]) if discharged_patients else np.nan

        summary_data[patient_type] = mean_length_of_stay  # Store in dictionary

    # Convert dictionary to DataFrame with a single row
    return pd.DataFrame([summary_data])

TRACE = False 

def single_run(scenario, rc_period=1445):
    ''' Runs the model for a single scenario and returns results as a DataFrame. '''
    
    env = simpy.Environment()
    model = AcuteStrokeUnit(env, scenario)

    # Start patient arrival processes
    env.process(model.stroke_arrivals_generator())
    env.process(model.tia_arrivals_generator())
    env.process(model.complexneuro_arrivals_generator())
    env.process(model.other_arrivals_generator())

    env.run(until=rc_period)  # Run simulation

    return summarize_results(model)  # Return DataFrame with results


In [15]:
# Create the default scenario
args = Scenario()

# Run the model
print('Running simulation ...', end=' => ')
results = single_run(args)
print('simulation complete.')

# Display the DataFrame
results


Running simulation ... => simulation complete.


,Other,ComplexNeuro,Stroke,TIA
0,3.814586,4.003766,7.365782,1.495207


In [16]:
# REPS

In [20]:
from joblib import Parallel, delayed
import pandas as pd
import numpy as np

def multiple_replications(scenario, rc_period=1445, n_reps=150, n_jobs=-1):
    """
    Run multiple independent replications of the model and return results in a DataFrame.
    Each row represents a single replication, and columns represent patient types.
    """
    # Run multiple replications in parallel
    res = Parallel(n_jobs=n_jobs)(
        delayed(single_run)(Scenario()) for _ in range(n_reps) 
    )
    # Concatenate results, keeping each replication as a row
    df_results = pd.concat(res, keys=range(1, n_reps + 1), names=["rep"])
    
    # Reset index to have 'rep' as a column
    df_results.reset_index(level=0, inplace=True)

    return df_results.set_index("rep")  # Final format with replications as rows

# Run multiple replications
args = Scenario()
print("Running multiple replications ... => ", end="")
results = multiple_replications(args, rc_period=1445, n_reps=150, n_jobs=-1)
print("Simulation complete.")

# Display the results
results

Running multiple replications ... => Simulation complete.


,Stroke,ComplexNeuro,Other,TIA
rep,,,,
1,7.365782,4.003766,3.814586,1.495207
2,7.365782,4.003766,3.814586,1.495207
3,7.365782,4.003766,3.814586,1.495207
4,7.365782,4.003766,3.814586,1.495207
5,7.365782,4.003766,3.814586,1.495207
...,...,...,...,...
146,7.365782,4.003766,3.814586,1.495207
147,7.365782,4.003766,3.814586,1.495207
148,7.365782,4.003766,3.814586,1.495207


In [21]:
results.mean()

Stroke          7.365782
ComplexNeuro    4.003766
Other           3.814586
TIA             1.495207
dtype: float64